<h2>Blum-Goldwasser Probabilistic Encryption</h2>

In [103]:
import numpy 
import math 
import random
import time

In [145]:
#Umbral a partir del cual consideramos que un numero es "grande"
umbral = 15
#Rango de valores a partir del umbral que se pueden generar
rango = 12

p, q, a, b = generarClavePrivada(umbral, rango)

clavePrivada = [p, q, a, b]
clavePublica = p*q

print("Clave privada --> ", clavePrivada)
print("Clave publica --> ", clavePublica)




Clave privada -->  [15, 27, -3, 0]
Clave publica -->  405


In [137]:
def generarClavePrivada(umbral, rango):
    p=0; q=0

    while not cumpleCongruencia(p, 3, 4):
        p = random.randint(umbral, umbral+rango)

    while not cumpleCongruencia(q, 3, 4)  or p==q: 
        q = random.randint(umbral, umbral+rango)

    a, b = calcularAlgEuclidesExtendido(p, q)

    return p, q, a, b

In [117]:
def cumpleCongruencia(num, numCongruente, modulo):
    return (num%modulo) == (numCongruente%modulo)

In [60]:
def calcularAlgEuclidesExtendido(p, q):
    restos = [p,q]
    listaA = [1,0]
    listaB = [0,1]
    cocientes = [0,0]

    i=2

    while(restos[i-1] > 1):
        cocientes.append(restos[i-2]//restos[i-1])
        restos.append(restos[i-2]%restos[i-1])
        listaA.append(listaA[i-2] - cocientes[i] * listaA[i-1])
        listaB.append(listaB[i-2] - cocientes[i] * listaB[i-1])
        i += 1

    if restos.pop()==0:
        return b, 0
    
    return listaA.pop(), listaB.pop()
